# Lending Club Data Credit Risk Analysis - Predizione dei Default



In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import sklearn.model_selection as model_selection
from dataCleaner import dataCleaner


In [2]:
data = pd.read_csv('loan.csv', low_memory=False)

print(data.shape)
data.head()

(39717, 111)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1077501,1296599,5000,5000,4975.0,36 months,10.65%,162.87,B,B2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27%,59.83,C,C4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96%,84.33,C,C5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49%,339.31,C,C1,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69%,67.79,B,B5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [3]:
data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

In [4]:
data.loan_status.value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

In [5]:
data = data[(data['loan_status'] == 'Fully Paid') | (data['loan_status'] == 'Charged Off')]

In [6]:
data.loan_status.value_counts()

Fully Paid     32950
Charged Off     5627
Name: loan_status, dtype: int64

In [7]:
data['loan_status'].head

<bound method NDFrame.head of 0         Fully Paid
1        Charged Off
2         Fully Paid
3         Fully Paid
5         Fully Paid
            ...     
39712     Fully Paid
39713     Fully Paid
39714     Fully Paid
39715     Fully Paid
39716     Fully Paid
Name: loan_status, Length: 38577, dtype: object>

Effettuiamo lo split training-test secondo la proporzione 70%-30%

In [8]:
columns = data.columns
features = columns.drop('loan_status')
target = ['loan_status']
x = data[features]
y = data[target]

train_x, test_x, train_y, test_y = model_selection.train_test_split(x, y, test_size=0.3, random_state=0)
dataCleaner = dataCleaner()

In [9]:

## FIX i warning
loan_status_dict = {'Fully Paid':0, 'Charged Off':1}
dataCleaner.set_target_mapDict(loan_status_dict)
dataCleaner.encode_target(train_y)


c:\Users\fabia\anaconda3\lib\site-packages\pandas\core\frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [10]:
train_y['loan_status'].head(10)

29845    0
31699    0
30480    0
35448    0
15977    0
29921    0
7628     0
28132    1
13205    0
34007    0
Name: loan_status, dtype: int64

In [11]:
#train_y.loan_status.astype(float) #TODO in teoria converto a float in dataCleaner.encode_target(train_y), ma non funziona, perché?

In [12]:
len(train_x.columns)

110

In [13]:
def unique_non_null(df):
    return df.dropna().unique()

In [14]:
colNotUniqueValue = []
for col in train_x.columns:
    colUniqueValuesCount = len(unique_non_null(data[col]))
    if colUniqueValuesCount > 1:
        colNotUniqueValue.append(col)
    print(col, "\t", colUniqueValuesCount)                  # DA FORMATTARE MEGLIO

id 	 38577
member_id 	 38577
loan_amnt 	 870
funded_amnt 	 1019
funded_amnt_inv 	 8050
term 	 2
int_rate 	 370
installment 	 15022
grade 	 7
sub_grade 	 35
emp_title 	 28027
emp_length 	 11
home_ownership 	 5
annual_inc 	 5215
verification_status 	 3
issue_d 	 55
pymnt_plan 	 1
url 	 38577
desc 	 25803
purpose 	 14
title 	 19297
zip_code 	 822
addr_state 	 50
dti 	 2853
delinq_2yrs 	 11
earliest_cr_line 	 524
inq_last_6mths 	 9
mths_since_last_delinq 	 95
mths_since_last_record 	 111
open_acc 	 40
pub_rec 	 5
revol_bal 	 21275
revol_util 	 1088
total_acc 	 82
initial_list_status 	 1
out_prncp 	 1
out_prncp_inv 	 1
total_pymnt 	 36714
total_pymnt_inv 	 36387
total_rec_prncp 	 6841
total_rec_int 	 34025
total_rec_late_fee 	 1320
recoveries 	 4040
collection_recovery_fee 	 2616
last_pymnt_d 	 101
last_pymnt_amnt 	 34418
next_pymnt_d 	 0
last_credit_pull_d 	 106
collections_12_mths_ex_med 	 1
mths_since_last_major_derog 	 0
policy_code 	 1
application_type 	 1
annual_inc_joint 	 0
dti_join

In [15]:
train_x = train_x.loc[:, colNotUniqueValue]

In [16]:
len(train_x.columns) 


44

Controlliamo se c'è qualche riga la cui percentuale di NaN supera il 50%

In [17]:
percentageNanInRow = np.ones(len(train_x))*len(train_x.columns)

percentageNanInRow = np.divide((train_x.isna().sum(axis=1)) , percentageNanInRow)
np.max(percentageNanInRow)

0.13636363636363635

Eliminazione delle features non utili alla predizione e separazione delle feature categoriche da quelle numeriche

In [18]:
categorical_features=[]
numerical_features=[]

In [19]:
colIdx = range(0,10)
train_x.iloc[:,colIdx].head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade
29845,511215,660289,9000,9000,9000.000000,36 months,10.25%,291.47,B,B2
31699,492140,629271,25000,25000,22725.000000,36 months,10.25%,809.62,B,B2
30480,507735,654945,5000,5000,5000.000000,36 months,15.33%,174.15,D,D3
35448,435440,520308,9000,9000,8748.151461,36 months,12.87%,302.70,C,C1
15977,739735,937342,5600,5600,5600.000000,60 months,19.36%,146.38,F,F4
29921,516506,667562,1500,1500,1500.000000,36 months,13.85%,51.16,C,C4
7628,879031,1093835,7500,7500,7425.000000,36 months,6.03%,228.27,A,A1
28132,543003,700619,4000,4000,3950.000000,60 months,11.12%,87.21,B,B3
13205,785977,989326,15000,15000,15000.000000,36 months,12.99%,505.34,C,C1
34007,438587,528605,14400,14400,14275.000000,36 months,11.14%,472.38,B,B1


<b>Attributi scartati:</b>

*id* e *member_id*: codici identificativi non utili per la predizione

*funded_amnt* e *funded_amnt_inv*: attributi che non sono disponibili al momento della scelta se effettuare il prestito o no

*grade* e *sub_grade*: attributi in stretta dipendenza con la feature *int_rate*


In [20]:
categorical_features.append('term')
numerical_features.append('loan_amnt')
numerical_features.append('installment')
numerical_features.append('int_rate')

In [21]:
colIdx = range(10,20)
train_x.iloc[:,colIdx].head(10)

,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,url,desc,purpose,title
29845,Loomis Group,3 years,RENT,55000.0,Not Verified,May-10,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Credit Cards
31699,Internal Revenue Service,10+ years,RENT,82000.0,Not Verified,Mar-10,https://lendingclub.com/browse/loanDetail.acti...,NaN,other,"50 down, 50 to go"
30480,Yahoo Inc.,3 years,RENT,94000.0,Not Verified,Apr-10,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 04/23/10 > The express pur...,debt_consolidation,Smartly Consolidating Debts for Lower Interest...
35448,Department of Energy,< 1 year,RENT,67092.0,Not Verified,Aug-09,https://lendingclub.com/browse/loanDetail.acti...,To Whom It May Concern: I have reached a stag...,major_purchase,Major Purchase
15977,Dootlitle's Woodfire,3 years,RENT,49008.0,Source Verified,May-11,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 04/27/11 > I am a chiropra...,major_purchase,Equinox
29921,Eglin Federal Credit Union,2 years,RENT,24000.0,Not Verified,May-10,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 05/12/10 > Loan Needed to ...,other,EDUCATION TROY 2010
7628,"Riverstone Enterprise Solutions, LLC",< 1 year,MORTGAGE,138000.0,Source Verified,Sep-11,https://lendingclub.com/browse/loanDetail.acti...,NaN,home_improvement,Bathroom Remodel
28132,Costco,1 year,MORTGAGE,24000.0,Source Verified,Jul-10,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,baby payoff loan
13205,Parish Anesthesia Consultants,5 years,MORTGAGE,370000.0,Verified,Jun-11,https://lendingclub.com/browse/loanDetail.acti...,Borrower added on 06/14/11 > This Loan is fo...,home_improvement,pool loan
34007,Mayer Brown LLP,2 years,OWN,115000.0,Not Verified,Nov-09,https://lendingclub.com/browse/loanDetail.acti...,528605 added on 11/09/09 > I am a responsibl...,debt_consolidation,Consolidation Loan


<b>Attributi scartati:</b>

*emp_title* e *issue_d*: attributi non utili per la predizione

*loan_status*: attributo sul quale di vuole effettuare la predizione

*url* e *desc*: attributi non utili per la predizione


In [22]:
categorical_features.append('home_ownership')
categorical_features.append('purpose')
categorical_features.append('verification_status')
numerical_features.append('emp_length')
numerical_features.append('annual_inc')

In [23]:
colIdx = range(20,30)
train_x.iloc[:,colIdx].head(10)

,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec
29845,941xx,CA,9.25,0,Dec-03,0,NaN,NaN,4,0
31699,018xx,MA,2.49,0,Jul-95,0,NaN,NaN,6,0
30480,920xx,CA,3.83,0,Jun-94,0,29.0,NaN,2,0
35448,208xx,MD,15.85,0,Nov-00,3,NaN,NaN,9,0
15977,551xx,MN,17.87,0,Jun-98,1,43.0,NaN,15,0
29921,325xx,FL,4.65,0,Oct-06,0,NaN,NaN,6,0
7628,210xx,MD,4.42,0,Aug-01,0,NaN,NaN,11,0
28132,871xx,NM,1.63,0,Apr-02,1,NaN,NaN,6,0
13205,707xx,LA,5.64,0,Oct-93,0,67.0,NaN,13,0
34007,606xx,IL,11.30,0,Feb-99,0,NaN,NaN,18,0


<b>Attributi scartati:</b>

*title*, *zip_code*, *addr_state*: attributi che risultano non essere correlati con la predizione

*mths_since_last_delinq* e *mths_since_last_record*: attributi che non sono disponibili al momento della scelta se effettuare il prestito o no

In [24]:
numerical_features.append('delinq_2yrs')
numerical_features.append('earliest_cr_line')
numerical_features.append('open_acc')
numerical_features.append('inq_last_6mths')
numerical_features.append('dti')

In [25]:
colIdx = range(30,len(train_x.columns))
train_x.iloc[:,colIdx].head(10)

,revol_bal,revol_util,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
29845,17031,58%,6,10493.412270,10493.41,9000.00,1493.41,0.00,0.0,0.0,May-13,350.83,May-13,0.0
31699,8185,8.30%,15,28326.220880,25748.54,25000.00,3326.22,0.00,0.0,0.0,Jan-12,3293.61,Jan-12,0.0
30480,4820,96.40%,14,6294.875422,6294.88,5000.00,1264.88,30.00,0.0,0.0,Mar-13,518.26,Dec-14,0.0
35448,22002,81.20%,18,10927.866280,10607.25,9000.00,1897.61,30.26,0.0,0.0,Sep-12,310.78,Oct-12,0.0
15977,5103,64.60%,25,7377.181049,7377.18,5600.00,1777.18,0.00,0.0,0.0,Oct-14,32.94,May-16,0.0
29921,3296,54.90%,8,1517.400000,1517.40,1500.00,17.40,0.00,0.0,0.0,Jul-10,1517.73,Mar-16,0.0
7628,3738,9.30%,19,8130.366066,8049.06,7500.00,630.37,0.00,0.0,0.0,Sep-13,2882.88,Oct-13,0.0
28132,3492,9.30%,11,2694.210000,2660.42,1785.03,909.18,0.00,0.0,0.0,Feb-13,87.21,May-16,0.0
13205,8078,68.50%,44,18192.110280,18192.11,15000.00,3192.11,0.00,0.0,0.0,Jul-14,526.33,Jul-14,0.0
34007,28541,41.30%,54,16815.431020,16669.46,14400.00,2415.43,0.00,0.0,0.0,Feb-12,4536.26,Dec-15,0.0


<b>Attributi scartati:</b>

*total_acc*, *total_pymnt* e *total_pymnt_inv*,

*total_rec_prncp*, *total_rec_int*, *total_rec_late_fee*: attributi che non sono disponibili al momento della scelta se effettuare il prestito o no

*recoveries* e *collection_recovery_fee*: attributi non utili per la predizione

*last_pymnt_d* e *last_pymnt_amnt*: attributi che non sono disponibili al momento della scelta se effettuare il prestito o no


In [26]:
numerical_features.append('pub_rec')
numerical_features.append('revol_util')
numerical_features.append('revol_bal')
numerical_features.append('last_credit_pull_d')
numerical_features.append('pub_rec_bankruptcies')

In [27]:
categorical_features

['term', 'home_ownership', 'purpose', 'verification_status']

In [28]:
numerical_features

['loan_amnt',
 'installment',
 'int_rate',
 'emp_length',
 'annual_inc',
 'delinq_2yrs',
 'earliest_cr_line',
 'open_acc',
 'inq_last_6mths',
 'dti',
 'pub_rec',
 'revol_util',
 'revol_bal',
 'last_credit_pull_d',
 'pub_rec_bankruptcies']

In [29]:
train_x_categorical=pd.DataFrame()
for col in categorical_features:
    train_x_categorical=pd.concat([ train_x_categorical, train_x[col]], axis= 1)
    dataCleaner.addFeatureToBeMantained(col)

train_x_categorical.head()

,term,home_ownership,purpose,verification_status
29845,36 months,RENT,debt_consolidation,Not Verified
31699,36 months,RENT,other,Not Verified
30480,36 months,RENT,debt_consolidation,Not Verified
35448,36 months,RENT,major_purchase,Not Verified
15977,60 months,RENT,major_purchase,Source Verified


In [30]:
train_x_numerical=pd.DataFrame()
for col in numerical_features:
    train_x_numerical=pd.concat([ train_x_numerical, train_x[col]], axis= 1)
    dataCleaner.addFeatureToBeMantained(col)

train_x_numerical.head()


,loan_amnt,installment,int_rate,emp_length,annual_inc,delinq_2yrs,earliest_cr_line,open_acc,inq_last_6mths,dti,pub_rec,revol_util,revol_bal,last_credit_pull_d,pub_rec_bankruptcies
29845,9000,291.47,10.25%,3 years,55000.0,0,Dec-03,4,0,9.25,0,58%,17031,May-13,0.0
31699,25000,809.62,10.25%,10+ years,82000.0,0,Jul-95,6,0,2.49,0,8.30%,8185,Jan-12,0.0
30480,5000,174.15,15.33%,3 years,94000.0,0,Jun-94,2,0,3.83,0,96.40%,4820,Dec-14,0.0
35448,9000,302.70,12.87%,< 1 year,67092.0,0,Nov-00,9,3,15.85,0,81.20%,22002,Oct-12,0.0
15977,5600,146.38,19.36%,3 years,49008.0,0,Jun-98,15,1,17.87,0,64.60%,5103,May-16,0.0


## Parsing delle colonne

### Feature Numeriche

In [31]:
#Rimozione del segno di percentuale dalle colonne 'revol_util' e 'int_rate'
#-----------------------------------------------------------------------------------------------------------------------------
featureList = ['revol_util', 'int_rate']
dataCleaner.removePercentageSign(train_x_numerical, featureList)

In [32]:
#Conversione delle date in numeri in quanto se gestite come variabili categoriche avrebbero generato troppe dummy variables
#------------------------------------------------------------------------------------------------------------------------------
featureList = ['last_credit_pull_d', 'earliest_cr_line']
dataCleaner.convertFromDateToInt(train_x_numerical, "%b-%y", featureList)

c:\Users\fabia\MachineLearningProject\dataCleaner.py:30: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  data[feature] = pd.DatetimeIndex(data[feature]).astype(np.int64)*1e-9


In [33]:
#Conversione di 'emp_length' in una variabile numerica ordinale
#-------------------------------------------------------------------------------------------------------------------------------
train_x_numerical.emp_length.unique()

array(['3 years', '10+ years', '< 1 year', '2 years', '1 year', '5 years',
       nan, '4 years', '6 years', '7 years', '8 years', '9 years'],
      dtype=object)

In [34]:
mapDict = {
    'emp_length': {
        '10+ years': 10,
        '9 years': 9,
        '8 years': 8,
        '7 years': 7,
        '6 years': 6,
        '5 years': 5,
        '4 years': 4,
        '3 years': 3,
        '2 years': 2,
        '1 year': 1,
        '< 1 year': 0,
    }
}

dataCleaner.set_emp_length_mapDict(mapDict)
dataCleaner.encode_emp_length(train_x_numerical)

In [35]:
train_x_numerical.head()

,loan_amnt,installment,int_rate,emp_length,annual_inc,delinq_2yrs,earliest_cr_line,open_acc,inq_last_6mths,dti,pub_rec,revol_util,revol_bal,last_credit_pull_d,pub_rec_bankruptcies
29845,9000,291.47,10.25,3.0,55000.0,0,1.070237e+09,4,0,9.25,0,58.0,17031,1.367366e+09,0.0
31699,25000,809.62,10.25,10.0,82000.0,0,8.045568e+08,6,0,2.49,0,8.3,8185,1.325376e+09,0.0
30480,5000,174.15,15.33,3.0,94000.0,0,7.704288e+08,2,0,3.83,0,96.4,4820,1.417392e+09,0.0
35448,9000,302.70,12.87,0.0,67092.0,0,9.730368e+08,9,3,15.85,0,81.2,22002,1.349050e+09,0.0
15977,5600,146.38,19.36,3.0,49008.0,0,8.966592e+08,15,1,17.87,0,64.6,5103,1.462061e+09,0.0


In [36]:
#Conversione delle variabili numeriche in float
#-------------------------------------------------------------------------------------------------------------------------------
dataCleaner.convertToFloat(train_x_numerical)

### Features Categoriche

In [37]:
for col in train_x_categorical.columns:
    print(col,'\n',train_x_categorical[col].unique(),'\n')

term 
 [' 36 months' ' 60 months'] 

home_ownership 
 ['RENT' 'MORTGAGE' 'OWN' 'OTHER' 'NONE'] 

purpose 
 ['debt_consolidation' 'other' 'major_purchase' 'home_improvement'
 'credit_card' 'house' 'car' 'medical' 'small_business' 'educational'
 'wedding' 'moving' 'vacation' 'renewable_energy'] 

verification_status 
 ['Not Verified' 'Source Verified' 'Verified'] 



In [38]:
#Creazione delle dummy variables tramite codifica One-Hot
#------------------------------------------------------------------------------------------------------------------------------
for col in train_x_categorical.columns:
    dataCleaner.addFeatureToBeOneHotEncoded(col)
   

train_x_categorical = dataCleaner.applyOneHotEncoding(train_x_categorical)

Index(['home_ownership', 'purpose', 'verification_status', ' 36 months'], dtype='object')
Index(['purpose', 'verification_status', ' 36 months', 'MORTGAGE', 'NONE',
       'OTHER', 'OWN'],
      dtype='object')
Index(['verification_status', ' 36 months', 'MORTGAGE', 'NONE', 'OTHER', 'OWN',
       'car', 'credit_card', 'debt_consolidation', 'educational',
       'home_improvement', 'house', 'major_purchase', 'medical', 'moving',
       'other', 'renewable_energy', 'small_business', 'vacation'],
      dtype='object')
Index([' 36 months', 'MORTGAGE', 'NONE', 'OTHER', 'OWN', 'car', 'credit_card',
       'debt_consolidation', 'educational', 'home_improvement', 'house',
       'major_purchase', 'medical', 'moving', 'other', 'renewable_energy',
       'small_business', 'vacation', 'Not Verified', 'Source Verified'],
      dtype='object')


In [39]:
train_x_categorical.head()


,36 months,MORTGAGE,NONE,OTHER,OWN,car,credit_card,debt_consolidation,educational,home_improvement,house,major_purchase,medical,moving,other,renewable_energy,small_business,vacation,Not Verified,Source Verified
29845,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
31699,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
30480,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
35448,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
15977,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
